# Deployment

In [9]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "langgraph-course"

## Testing Locally

In [2]:
from langgraph_sdk import get_client

In [4]:
# Replace this with the URL of your own deployed graph
URL = "http://localhost:63330"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [6]:
# langgraph studio is running in the background
assistants

[{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'created_at': '2024-09-14T20:37:26.896330+00:00',
  'updated_at': '2024-09-14T20:37:26.896330+00:00',
  'config': {},
  'metadata': {'created_by': 'system'}},
 {'assistant_id': '228f9934-0cdd-5383-92c8-ee8422522cc2',
  'graph_id': 'router',
  'created_at': '2024-09-14T19:26:49.051121+00:00',
  'updated_at': '2024-09-14T19:26:49.051121+00:00',
  'config': {},
  'metadata': {'created_by': 'system'}}]

In [7]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

Now, we can run our agent [with `client.runs.stream`](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream) with:

* The `thread_id`
* The `graph_id`
* The `input` 
* The `stream_mode`

We'll discuss streaming in depth in a future module. 

For now, just recognize that we are [streaming](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/) the full value of the state after each step of the graph with `stream_mode="values"`.
 
The state is captured in the `chunk.data`. 

In [10]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'c300a63f-2a2f-4ed1-945b-f8f9da2b8811', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_8uf3ePxer8ZV6A9JqLf6qgv4', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5'}, 'type': 'ai', 'name': None, 'id': 'run-7b0de742-8265-43d5-94db-aafb6b7edb7c', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_8uf3ePxer8ZV6A9JqLf6qgv4', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '4b6eeb13-ecde-4118-8195-973ef48da072', 'tool_call_id': 'c